In [1]:
import pandas as pd 
import numpy as np 
from hazm import Normalizer, WordTokenizer, Lemmatizer, WordEmbedding
import string 
import re

In [2]:
df = pd.read_csv("emails.csv")

In [3]:
df

,text,label
0,﻿ممنون آقا سامان.\nمن پارسال اصلا آزاد شرکت نک...,ham
1,﻿سلام آقای کریمی\nبالاخره آزمونارشد تموم شد من...,ham
2,﻿درود بر حاج وحیدی بنده بعنوان یک دکتری تاریخ ...,ham
3,﻿با سلام و احترام\nضمن تقدیر از مسولین محترم ...,ham
4,﻿با سلام اینجانب یک دستگاه خودرو پراید 131 با ...,ham
...,...,...
995,﻿\nبسمه تعالی\n\nسازمان زیباسازی شهرداری استان...,spam
996,﻿\n\nبه مناسبت فرا رسیدن میلاد دخت پیامبر گرام...,spam
997,﻿\nدرود هموطن من\n\n \n\nتست رایگان \n\n ...,spam
998,﻿\n\n *درج **لینک در 8700 وبلاگ\n *\n\n...,spam


## Data preprocessing 

### Removing '\n's

In [4]:
# Remove the "\n" characters from the text column
df['text'] = df['text'].str.replace('\n', ' ')


### Removing punctuation 

In [5]:
# Function to remove punctuation from text
def remove_punctuation(text):
    # Define a translation table with all punctuation characters mapped to None
    translator = str.maketrans(' ', ' ', string.punctuation)
    # Remove punctuation using the translation table
    return text.translate(translator)

# Apply the function to the "Content" column
df['text'] = df['text'].apply(remove_punctuation)


### Removing numbers 

In [6]:
# Function to remove numbers from text using regular expressions
def remove_numbers(text):
    # Use regular expression to remove all numbers
    return re.sub(r'\d+', '', text)

df['text'] = df['text'].apply(remove_numbers)

In [7]:
normalizer = Normalizer()
df['text'] = df['text'].apply(lambda text: normalizer.normalize(text))

In [8]:
tokenizer = WordTokenizer()
df['Tokenized_Content'] = df['text'].apply(lambda text: tokenizer.tokenize(text))

In [9]:
# Read stop words from the text file
with open("PersianStopWords.txt", "r", encoding="utf-8") as file:
    stop_words = set(file.read().splitlines())

# Define a function to remove stop words
def remove_stop_words(tokens):
    return [word for word in tokens if word not in stop_words]

# Apply the function to the "Tokenized_Content" column
df['Tokenized_Content'] = df['Tokenized_Content'].apply(remove_stop_words)

In [10]:
df

,text,label,Tokenized_Content
0,﻿ممنون آقا سامان من پارسال اصلا آزاد شرکت نکرد...,ham,"[﻿ممنون, آقا, سامان, آزاد, شرکت, نکرده_بودم, س..."
1,﻿سلام آقای کریمی بالاخره آزمونارشد تموم شد من ...,ham,"[﻿سلام, آقای, کریمی, آزمونارشد, تموم, شدم, یکم..."
2,﻿درود بر حاج وحیدی بنده بعنوان یک دکتری تاریخ ...,ham,"[﻿درود, حاج, وحیدی, بنده, بعنوان, یک, دکتری, ت..."
3,﻿با سلام و احترام ضمن تقدیر از مسولین محترم سا...,ham,"[﻿با, سلام, احترام, ضمن, تقدیر, مسولین, محترم,..."
4,﻿با سلام اینجانب یک دستگاه خودرو پراید با شمار...,ham,"[﻿با, سلام, اینجانب, یک, دستگاه, خودرو, پراید,..."
...,...,...,...
995,﻿ بسمه تعالی سازمان زیباسازی شهرداری استان تهر...,spam,"[﻿, بسمه, تعالی, سازمان, زیباسازی, شهرداری, اس..."
996,﻿ به مناسبت فرا رسیدن میلاد دخت پیامبر گرامی ا...,spam,"[﻿, مناسبت, فرا, رسیدن, میلاد, دخت, پیامبر, گر..."
997,﻿ درود هموطن من تست رایگان تحویل اکانت پرداخت ...,spam,"[﻿, درود, هموطن, تست, رایگان, تحویل, اکانت, پر..."
998,﻿ درج لینک در وبلاگ درج لینک و تبلیغات متنی شم...,spam,"[﻿, درج, لینک, وبلاگ, درج, لینک, تبلیغات, متنی..."


In [12]:
df['Tokenized_Content'] = df['Tokenized_Content'].apply(lambda tokens: ' '.join(tokens))

In [14]:
wordEmbedding = WordEmbedding(model_type = 'fasttext')

In [15]:
text = df["text"]